# 2D Axi-symmetric Actuator

This example demonstrates how to leverage both: axi-symmetry and the magnetostatic solver
to calculate the forces experienced by the anchor of an actuator due to change in
current and anchor location

Keywords: **Maxwell2D**, **axi-symmetry** **magnetostatic**, **translational motion**, **parametric sweep**,
**installation example**

## Prerequisites

### Perform imports

In [1]:
import os
import tempfile
import time

import ansys.aedt.core  # Interface to Ansys Electronics Desktop


### Define constants
Constants help ensure consistency and avoid repetition throughout the example.

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

### Create temporary directory

Create a temporary working directory.
The name of the working folder is stored in ``temp_folder.name``.

> **Note:** The final cell in the notebook cleans up the temporary folder. If you want to
> retrieve the AEDT project and data, do so before executing the final cell in the notebook.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

### Launch Maxwell 2d

Create an instance of
the ``Maxwell2d`` class. The Ansys Electronics Desktop will be launched
with an active Maxwell2D design. The ``m2d`` object is subsequently
used to
create and simulate the actuator model.

In [4]:
project_name = os.path.join(temp_folder.name, "2d_axi_magsta_actuator.aedt")
m2d = ansys.aedt.core.Maxwell2d(
    project=project_name,
    solution_type="MagnetostaticZ",
    version=AEDT_VERSION,
    non_graphical=NG_MODE,
    new_desktop=True,
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_0da1e1a3-9a2a-44a7-8fd7-0c23400096c1.log is enabled.


...

PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project 2d_axi_magsta_actuator has been created.


PyAEDT INFO: No design is present. Inserting a new design.


PyAEDT INFO: Added design 'Maxwell 2D_THY' of type Maxwell 2D.


PyAEDT INFO: Aedt Objects correctly read


## Model Preparation

### Define actuator's housing point location

The actuator's housing is built using line segments
for readability and reusability we define the points
as a python's list of lists, where each sublist defines an
x-, y-, z-coordinate. (e.g., [[x1,y1,z1],...,[xn,yn,zn]].

In [5]:
points_housing = [
    [0, 0, 0],
    [0, 0, -10],
    [12, 0, -10],
    [12, 0, 10],
    [2.5, 0, 10],
    [2.5, 0, 8],
    [10, 0, 8],
    [10, 0, -8],
    [2, 0, -8],
    [2, 0, 0],
]

### Declare and initialize design parameters

In [6]:
m2d.modeler.model_units = "mm"  # Global units used in geometry creation
m2d["Amp_1"] = "1000A"  # Net current applied to coil
m2d["move"] = "0mm"  # Displacement applied to anchor

PyAEDT INFO: Modeler2D class has been initialized!


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


### Create 2D model
Build coil, anchor and housing geometries

In [7]:
coil_m = m2d.modeler.create_rectangle(origin=["3mm", "0mm", "7mm"], sizes=[-14, 6], name="Coil", material="Copper")
anchor_m = m2d.modeler.create_rectangle(
    origin=["0mm", "0mm", "13mm - move"],
    sizes=[-8, 2],
    name="Anchor",
    material="steel_1008",
)
housing_m = m2d.modeler.create_polyline(points_housing, close_surface=True, name="Housing", material="steel_1008")
m2d.modeler.cover_lines(housing_m)

PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


True

Create surrounding vacuum domain

In [8]:
region_m = m2d.modeler.create_region(pad_percent=100)
region_m.material_name = "vacuum"

Fit all geometrical entities into the modeler's window

In [9]:
m2d.modeler.fit_all()

### Assign boundary conditions

Apply zero magnetic vector potential on the region edges
that the field only has a tangential component at the edge of the
surrounding domain/region

In [10]:
m2d.assign_vector_potential(assignment=region_m.edges, boundary="VectorPotential1")

PyAEDT INFO: Boundary Vector Potential VectorPotential1 has been created.


VectorPotential1

### Assign Excitation

Create a current driven coil by applying a current excitation in the coil domain

In [11]:
m2d.assign_current(assignment=coil_m.name, amplitude="Amp_1", name="Current1")

PyAEDT INFO: Boundary Current Current1 has been created.


Current1

### Enable force calculation on the anchor

In [12]:
m2d.assign_force(
    assignment=anchor_m,
    force_name="Force",
)

PyAEDT INFO: Boundary Force Force has been created.


Force

### Define solution setup

In [13]:
setup = m2d.create_setup("MySetup")
print(setup.props)
setup.props["MaximumPasses"] = 3
setup.props["PercentRefinement"] = 30
setup.props["PercentError"] = 0.1
setup.props["MinimumPasses"] = 1
setup.props["RelativeResidual"] = 1e-6

{'Enabled': True, 'MeshLink': {'ImportMesh': False}, 'MaximumPasses': 10, 'MinimumPasses': 2, 'MinimumConvergedPasses': 1, 'PercentRefinement': 30, 'SolveFieldOnly': False, 'PercentError': 1, 'SolveMatrixAtLast': True, 'UseIterativeSolver': False, 'RelativeResidual': 1e-06, 'NonLinearResidual': 0.001, 'SmoothBHCurve': False, 'MuOption': {'MuNonLinearBH': True}, 'Name': 'MySetup'}


### Create variable/parameter sweeps

Enable sweeps over coil net current and anchor displacement

In [14]:
value_sweep = m2d.parametrics.add("Amp_1", 500, 1000, 500, name="ParametricSetup1", variation_type="LinearStep")
value_sweep.add_variation("move", 0, 2, 1, variation_type="LinearStep")
#

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpqv5301qh.ansys\2d_axi_magsta_actuator.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpqv5301qh.ansys\2d_axi_magsta_actuator.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.015713214874267578


True

### Run analysis

In [15]:
value_sweep.analyze(cores=NUM_CORES)

PyAEDT INFO: Project 2d_axi_magsta_actuator Saved correctly


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


PyAEDT INFO: Solving Optimetrics


PyAEDT INFO: Design setup ParametricSetup1 solved correctly in 0.0h 0.0m 28.0s


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


True

## Postprocess

Create a Force vs. move (displacement) Report/Plot for every current value

In [16]:
m2d.post.create_report(
    expressions=["Force.Force_z"],
    variations={"Amp_1": "All", "move": "All"},
    plot_name="Force Plot 1",
    primary_sweep_variable="move",
    plot_type="Rectangular Plot",
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpqv5301qh.ansys\2d_axi_magsta_actuator.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpqv5301qh.ansys\2d_axi_magsta_actuator.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.03164792060852051


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


PyAEDT WARNING: No report category provided. Automatically identified Magnetostatic


Force Plot 1

## Finish

### Save the project

In [17]:
m2d.save_project()
m2d.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project 2d_axi_magsta_actuator Saved correctly


PyAEDT INFO: Desktop has been released and closed.


### Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [18]:
temp_folder.cleanup()